In [ ]:
Resources
| where type == 'microsoft.keyvault/vaults'
| project name, type, location, subscriptionId, resourceGroup
| join (ResourceContainers | where type=='microsoft.resources/subscriptions' | project SubName=name, subscriptionId) on subscriptionId
| project type, name, location, resourceGroup, SubName
| limit 1

In [ ]:
resources
| mv-expand ipConfigurations=properties.ipConfigurations
| mv-expand ip=ipConfigurations.properties.privateIPAddress
| where isnotnull(ip)
| extend virtualMachineID=tostring(properties.virtualMachine.id)
| join kind=leftouter (
    resources
    | where type =~ 'microsoft.compute/virtualmachines'
    | project virtualMachineID = id,VM_Name=name,VM_resourceGroup=resourceGroup,VM_subscriptionId=subscriptionId)
    on virtualMachineID
| project-away virtualMachineID1
| join kind=leftouter(
    resources
    | project managedBy = id, PEndpoint_Name=name,PEndpoint_resourceGroup=resourceGroup,PEndpoint_subscriptionId=subscriptionId)
    on managedBy
| project-away managedBy1
| project-away tenantId,kind
//| project ['id'],type,virtualMachine,properties.ipConfigurations,ip